In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import json
import random
from urllib.parse import urlparse
import logging
import pandas as pd
from transformers import pipeline
from PIL import Image
import numpy as np
import os
import torch
from transformers import AutoImageProcessor, AutoModelForObjectDetection

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(message)s')

# Base URL components
# BASE_URL = "https://www.si.edu/search/collection-images?edan_q=&edan_fq%5B0%5D=topic%3A%22Costume%22"
BASE_URL = "https://www.si.edu/search/collection-images?edan_q=&edan_fq%5B%5D=media_usage:%22CC0%22&edan_fq%5B%5D=topic:%22Costume%22+OR+topic%3A%22Dress+accessories%22+OR+topic%3A%22Hats%22+OR+topic%3A%22Headgear%22+OR+topic%3A%22Neckties%22+OR+topic%3A%22Neckwear%22"


USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko)",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko)",
    # Add more user agents as needed
]

# Function to construct URL with page number
def construct_url(page_number):
    if page_number > 1:
        # Insert page parameter after collection-images? for subsequent pages
        page_param = f"page={page_number - 1}&"
        url_parts = BASE_URL.split("?")
        return f"{url_parts[0]}?{page_param}{url_parts[1]}"
    return BASE_URL

# Function to extract item links from a page
def get_item_links(soup):
    item_links = []
    for li in soup.find_all("li", attrs={"ogmt-id": True}):
        a_tag = li.find("a", class_="inner")
        if a_tag and 'href' in a_tag.attrs:
            item_links.append(a_tag['href'])
    return item_links

from urllib.parse import urlparse, parse_qs, urlencode

def extract_item_info(session, item_url, item_number=None):
    full_url = f"https://www.si.edu{item_url}"
    try:
        response = session.get(full_url, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        logging.warning(f"Request exception for item {item_number}: {full_url}: {e}")
        return None

    content = response.content
    logging.info(f"Retrieving item {item_number}: {full_url}")

    soup = BeautifulSoup(content, 'html.parser')

    # Extract the h1 tag
    h1_tag = soup.find("h1")
    if not h1_tag:
        logging.warning(f"No h1 tag found in {full_url}")
        return None
    h1_text = h1_tag.get_text(strip=True)

    # Initialize the item dictionary
    item_dict = {'Title': h1_text}

    # Extract the "Museum" information from the next <a> tag after <h1>
    next_a_tag = h1_tag.find_next("a")
    if next_a_tag and next_a_tag.get_text(strip=True):
        museum_name = next_a_tag.get_text(strip=True)
        item_dict["Museum"] = museum_name
    else:
        logging.warning(f"No Museum information found in {full_url}")

    # Iterate through all <dl> tags directly
    for dl in soup.find_all("dl"):
        current_key = None
        values = []
        for child in dl.children:
            if child.name == "dt":
                if current_key and values:
                    # Assign the collected values to the previous key
                    item_dict[current_key] = values
                # Start a new key
                current_key = child.get_text(strip=True)
                values = []
            elif child.name == "dd":
                dd_text = child.get_text(strip=True)
                values.append(dd_text)
        # After the loop, assign the last collected values
        if current_key and values:
            item_dict[current_key] = values

    # Try to extract the 'Screen Image' link
    screen_image_link = soup.find('a', text='Screen Image')
    if screen_image_link and screen_image_link.has_attr('href'):
        image_url = screen_image_link['href']
        if not image_url.startswith('http'):
            image_url = f"https://ids.si.edu{image_url}"
        item_dict["Image_URL"] = image_url
        logging.info(f"    Screen Image URL extracted: {image_url}")
    else:
        # Fallback method: extract from <img id='edan-image'> tag
        img_tag = soup.find('img', id='edan-image')
        if img_tag and img_tag.has_attr('src'):
            img_src = img_tag['src']
            # Parse the URL
            parsed_url = urlparse(img_src)
            query_params = parse_qs(parsed_url.query)
            # Append '.jpg' to the 'id' parameter
            if 'id' in query_params:
                id_value = query_params['id'][0]
                if not id_value.endswith('.jpg'):
                    id_value += '.jpg'
                query_params['id'] = [id_value]
                # Reconstruct the query string
                new_query = urlencode(query_params, doseq=True)
                # Reconstruct the full URL
                image_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}?{new_query}"
                item_dict["Image_URL"] = image_url
                logging.info(f"    Image URL extracted from img tag: {image_url}")
            else:
                logging.warning(f"    No 'id' parameter in img src for {full_url}")
        else:
            logging.warning(f"    No valid Image URL found in {full_url}")

    return item_dict

def scrape_smithsonian_collection(total_pages=3, checkpoint_dir='checkpoints', resume=False):
    # Create checkpoint directory
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Find last checkpoint if resuming
    start_page = 1
    if resume:
        checkpoints = sorted([f for f in os.listdir(checkpoint_dir) if f.endswith('.pkl')])
        if checkpoints:
            last_checkpoint = checkpoints[-1]
            start_page = int(last_checkpoint.split('_')[1].split('.')[0]) + 1
            # Load last checkpoint data
            all_items = pd.read_pickle(os.path.join(checkpoint_dir, last_checkpoint)).to_dict('records')
        else:
            all_items = []
    else:
        all_items = []

    total_scraped = len(all_items)
    
    # Initialize session
    session = requests.Session()
    session.headers.update({"User-Agent": random.choice(USER_AGENTS)})

    # Start scraping from last checkpoint
    for page in range(start_page, total_pages + 1):
        logging.info(f"Scraping Page {page}")
        
        try:
            # Existing scraping logic
            url = construct_url(page)
            response = session.get(url, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            item_links = get_item_links(soup)
            
            page_items = []
            for idx, item_link in enumerate(item_links, 1):
                item_info = extract_item_info(session, item_link)
                if item_info:
                    page_items.append(item_info)
                    total_scraped += 1
                time.sleep(random.uniform(1, 3))
            
            # Add page items to main list
            all_items.extend(page_items)
            
            # Save checkpoint after each page
            df = pd.DataFrame(all_items)
            checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_{page}.pkl')
            df.to_pickle(checkpoint_path)
            logging.info(f"Saved checkpoint for page {page}")
            
        except Exception as e:
            logging.error(f"Error on page {page}: {e}")
            break
    
    final_df = pd.DataFrame(all_items)
    final_df.to_pickle('final_data.pkl')
    return final_df

# def scrape_smithsonian_collection(total_pages=3):
#     all_items = []
#     total_scraped = 0

#     # Initialize a session
#     session = requests.Session()
#     session.headers.update({
#         "User-Agent": random.choice(USER_AGENTS)
#     })

#     for page in range(1, total_pages + 1):
#         url = construct_url(page)
#         logging.info(f"Scraping Page {page}: {url}")
#         try:
#             response = session.get(url, timeout=10)
#             response.raise_for_status()
#         except requests.exceptions.RequestException as e:
#             logging.warning(f"Request exception for page {page}: {e}")
#             continue

#         soup = BeautifulSoup(response.content, 'html.parser')
#         item_links = get_item_links(soup)
#         logging.info(f"Found {len(item_links)} items on Page {page}")

#         for idx, item_link in enumerate(item_links, 1):
#             logging.info(f"  Scraping Item {idx}: {item_link}")
#             item_info = extract_item_info(session, item_link)
#             if item_info:
#                 all_items.append(item_info)
#                 total_scraped += 1
#             else:
#                 logging.warning(f"    Failed to extract info for {item_link}")

#             # Add a random pause between 1 to 3 seconds
#             sleep_time = random.uniform(.5, 2.5)
#             logging.info(f"    Sleeping for {sleep_time:.2f} seconds...")
#             time.sleep(sleep_time)

#         logging.info(f"Completed scraping Page {page}\n")

#     logging.info("Scraping completed.")
#     # Convert list of dicts to DataFrame
#     df = pd.DataFrame(all_items)
#     return df

# Load the NER pipeline
from transformers import pipeline

token_classifier = pipeline(
    "ner", model="dslim/bert-base-NER", aggregation_strategy="simple"
)

def extract_entities(df, entity_score_threshold=0.8):
    """
    Extracts named entities from the DataFrame's text columns and adds them as new columns.

    Parameters:
    - df (pandas.DataFrame): The DataFrame containing scraped data.
    - entity_score_threshold (float): The minimum confidence score for entities to be considered.

    Returns:
    - df (pandas.DataFrame): The updated DataFrame with new entity columns.
    """
    import pandas as pd
    
    # Ensure the DataFrame index is unique
    if not df.index.is_unique:
        df = df.reset_index(drop=True)
    
    # Initialize a dictionary to hold the new columns
    new_columns = {}
    
    # Iterate over each row in the DataFrame
    for idx, row in df.iterrows():
        # Dictionary to hold entities for this row
        row_entities = {}
        
        # Iterate over each column in the row
        for column in df.columns:
            # Skip columns that should not be processed for NER
            if column in ['Title', 'Museum', 'Image_URL', 'cropped_objects']:
                continue
            
            value = row[column]
            
            # Initialize list to hold text snippets to process
            texts_to_process = []
            
            # Determine how to handle the value based on its type
            if isinstance(value, list):
                # If it's a list, check if all elements are null
                if all(pd.isnull(v) for v in value):
                    continue  # Skip if all elements are null
                # Filter out non-string elements
                texts_to_process = [str(v) for v in value if isinstance(v, str)]
            elif isinstance(value, str):
                if pd.isnull(value):
                    continue  # Skip if the string is null
                texts_to_process = [value]
            else:
                # For any other type (e.g., NaN, None), skip processing
                continue
            
            # If there are no texts to process, skip to the next column
            if not texts_to_process:
                continue
            
            # Process each text snippet
            for text in texts_to_process:
                # Run the NER model on the text
                try:
                    results = token_classifier(text)
                except Exception as e:
                    logging.warning(f"NER model failed on text '{text}': {e}")
                    continue  # Skip this text snippet if NER fails
                
                # Filter entities based on the score threshold
                filtered_results = [entity for entity in results if entity['score'] >= entity_score_threshold]
                
                # Iterate over filtered entities and organize them by entity group
                for entity in filtered_results:
                    entity_group = entity['entity_group']
                    word = entity['word'].replace('\n', ' ').strip()  # Clean up the word
                    
                    # Create a new column name based on the original column and entity group
                    col_name = f"{column}.{entity_group}"
                    
                    # Initialize the list for this entity group if not already present
                    if col_name not in row_entities:
                        row_entities[col_name] = []
                    
                    # Append the extracted word to the list
                    row_entities[col_name].append(word)
        
        # Store the entities for this row
        for col_name, words in row_entities.items():
            if col_name not in new_columns:
                new_columns[col_name] = [None] * len(df)
            new_columns[col_name][idx] = words
    
    # After processing all rows, add the new columns to the DataFrame
    for col_name, column_data in new_columns.items():
        df[col_name] = column_data
    
    return df

# Load the processor and model for object detection
processor = AutoImageProcessor.from_pretrained("valentinafeve/yolos-fashionpedia")
model = AutoModelForObjectDetection.from_pretrained("valentinafeve/yolos-fashionpedia")

def analyze_and_crop_image(image_path, image_id, output_dir, category_data):
    """
    Analyzes an image using the Fashionpedia model, saves cropped images, and returns a dictionary of objects data.
    """
    from PIL import Image
    import torch
    import numpy as np
    
    # Load category mappings
    categories = category_data['categories']
    category_id_to_name = {category['id']: category['name'] for category in categories}
    category_id_to_supercategory = {category['id']: category['supercategory'] for category in categories}
    label_mappings = model.config.id2label  # This maps label IDs to category names
    name_to_supercategory = {category['name']: category['supercategory'] for category in categories}
    
    # Load the image
    image = Image.open(image_path)
    # Ensure image is in RGB format
    if image.mode != 'RGB':
        image = image.convert('RGB')
    # Convert the image to a NumPy array
    image_array = np.array(image)
    
    # Preprocess the image
    inputs = processor(images=image_array, return_tensors="pt")
    # Run inference
    outputs = model(**inputs)
    # Post-process the outputs to get object detection results
    results = processor.post_process_object_detection(
        outputs, threshold=0.5, target_sizes=[image.size[::-1]]
    )
    result = results[0]
    
    objects_data = {}
    # Get image dimensions
    width, height = image.size
    
    # Loop over each detection
    for idx, (score, label_id, box) in enumerate(zip(result['scores'], result['labels'], result['boxes'])):
        score = score.item()
        label_id = label_id.item()
        box = box.tolist()
        x_min, y_min, x_max, y_max = map(int, box)
        area = (x_max - x_min) * (y_max - y_min)
        name = label_mappings.get(label_id, 'Unknown')
        supercategory = name_to_supercategory.get(name, 'Unknown')
        category_id = next((id for id, n in category_id_to_name.items() if n == name), label_id)
        
        # Save the cropped image
        # Clip coordinates to image bounds
        x_min = max(0, min(width, x_min))
        y_min = max(0, min(height, y_min))
        x_max = max(0, min(width, x_max))
        y_max = max(0, min(height, y_max))
        
        # Check for valid crop
        if x_max > x_min and y_max > y_min:
            # Crop the image
            cropped_image = image.crop((x_min, y_min, x_max, y_max))
            # Ensure cropped image is in RGB mode
            if cropped_image.mode != 'RGB':
                cropped_image = cropped_image.convert('RGB')
            # Create a unique filename
            cropped_image_name = f"{os.path.splitext(os.path.basename(image_path))[0]}_{name.replace(' ', '_')}_{idx}.jpg"
            cropped_image_path = os.path.join(output_dir, cropped_image_name)
            try:
                # Save the cropped image
                cropped_image.save(cropped_image_path, format='JPEG')
                print(f"Saved cropped image: {cropped_image_path}")
            except Exception as e:
                print(f"Failed to save {cropped_image_path}: {e}")
                continue
        else:
            print(f"Skipping invalid crop for detection {idx}")
            continue
        
        # Collect the data
        object_key = f"Object_{idx+1}"
        objects_data[object_key] = {
            'Name': name,
            'Supercategory': supercategory,
            'Bounding Box': box,
            'Area': area,
            'cropped_image_path': cropped_image_path
        }
    return objects_data

def process_images(df, output_dir, category_json_path):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Load category mappings
    with open(category_json_path, 'r') as f:
        category_data = json.load(f)
    
        object_rows = []
        for idx, row in df.iterrows():
            image_url = row['Image_URL']
            image_name = f"image_{idx}.jpg"
            image_path = os.path.join(output_dir, image_name)
            try:
                response = requests.get(image_url, stream=True)
                response.raise_for_status()
                with open(image_path, 'wb') as out_file:
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            out_file.write(chunk)
            except requests.exceptions.RequestException as e:
                print(f"Failed to download image {image_url}: {e}")
                continue
            
            # Process the image
            try:
                objects_data = analyze_and_crop_image(image_path, idx, output_dir, category_data)
                for object_id, object_info in objects_data.items():
                    # Add image index or any other relevant info
                    object_info['Image_Index'] = idx
                    # Include the object ID
                    object_info['Object_ID'] = object_id
                    object_rows.append(object_info)
            except Exception as e:
                print(f"Failed to process image {image_path}: {e}")
                continue

    # Create a DataFrame where each row is an object
    objects_df = pd.DataFrame(object_rows)
    # Set 'Object_ID' as the index if desired
    objects_df.set_index('Object_ID', inplace=True)
    return objects_df

# if __name__ == "__main__":
#     # Scrape the data
#     scraped_df = scrape_smithsonian_collection(total_pages=2)
#     print(f"Scraping Completed. Total items scraped: {len(scraped_df)}")
#     # Save the scraped data to a CSV file
#     scraped_df.to_csv('scraped_data.csv', index=False)


if __name__ == "__main__":

    # Define scraping parameters
    TOTAL_PAGES = 2
    CHECKPOINT_DIR = 'checkpoints'
    RESUME = True  # Set to True to resume from last checkpoint

    # Scrape the data with checkpointing
    scraped_df = scrape_smithsonian_collection(
        total_pages=TOTAL_PAGES,
        checkpoint_dir=CHECKPOINT_DIR,
        resume=RESUME
    )
    
    print(f"Scraping Completed. Total items scraped: {len(scraped_df)}")
    # Extract entities
    print("Extracting entities...")
    scraped_df = extract_entities(scraped_df, entity_score_threshold=0.8)
    # Save the data with entities to a CSV file
    scraped_df.to_csv('scraped_data_with_entities.csv', index=False)
    
    # Process images
    print("Processing images...")
    scraped_df = process_images(scraped_df, output_dir='archive/cropped_images', category_json_path='/Users/joshstrupp/Documents/Working/Educational/MSDV/ms1-final/category_attributes_descriptions.json')
    # Save the final DataFrame to a CSV file
    scraped_df.to_csv('final_data.csv', index=False)
    # Optionally, save the DataFrame to a pickle file
    scraped_df.to_pickle('final_data.pkl')

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Scraping Page 1
Retrieved https://www.si.edu/object/william-caxton-examining-first-proof-sheet-his-printing-press-westminster-abbey-ad-1474:saam_1966.48.8
/var/folders/pc/j_zmpc9s0g9frv2jzfv1l

Scraping Completed. Total items scraped: 108
Extracting entities...
Processing images...
Saved cropped image: archive/cropped_images/image_1_skirt_0.jpg
Saved cropped image: archive/cropped_images/image_3_hat_0.jpg
Saved cropped image: archive/cropped_images/image_3_top,_t-shirt,_sweatshirt_1.jpg
Saved cropped image: archive/cropped_images/image_3_sleeve_2.jpg
Saved cropped image: archive/cropped_images/image_3_neckline_3.jpg
Saved cropped image: archive/cropped_images/image_3_sleeve_4.jpg
Saved cropped image: archive/cropped_images/image_6_hat_0.jpg
Saved cropped image: archive/cropped_images/image_6_sleeve_1.jpg
Saved cropped image: archive/cropped_images/image_7_sleeve_0.jpg
Saved cropped image: archive/cropped_images/image_8_sleeve_0.jpg
Saved cropped image: archive/cropped_images/image_9_pocket_0.jpg
Saved cropped image: archive/cropped_images/image_9_collar_1.jpg
Saved cropped image: archive/cropped_images/image_9_pocket_2.jpg
Saved cropped image: archive/cropped_images/image_9_p